In [1]:
from lmu import FeedForwardCell
from tensorflow.keras.layers import RNN, Dense, Dropout
from tensorflow.keras import Sequential
import tensorflow as tf
import numpy as np
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
FFCell = FeedForwardCell(100, 784)
layer = RNN(FFCell, input_shape=(784, 1), return_sequences=False)

model = Sequential()
model.add(layer)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn_1 (RNN)                  (None, 100)               30401     
Total params: 30,401
Trainable params: 0
Non-trainable params: 30,401
_________________________________________________________________


In [4]:
tf.random.set_seed(0)

(
    (train_images, train_labels),
    (test_images, test_labels),
) = tf.keras.datasets.mnist.load_data()


def normalize(x):
    return (x - np.mean(x)) / np.std(x)


train_images = normalize(train_images)
test_images = normalize(test_images)

train_images = train_images.reshape((train_images.shape[0], -1, 1))
test_images = test_images.reshape((test_images.shape[0], -1, 1))

# apply permutation
rng = np.random.RandomState(0)
perm = rng.permutation(train_images.shape[1])
train_images = train_images[:, perm]
test_images = test_images[:, perm]

index = np.random.choice(train_images.shape[0], 1000, replace=False)

In [5]:
start = time.time()
m_train = model.predict(train_images[index])
end = time.time()
print(end-start)

1.900804042816162


In [6]:
model_ff = Sequential()
model_ff.add(Dense(50, input_shape=(100,), activation='tanh'))
model_ff.add(Dropout(0.5))
model_ff.add(Dense(10, activation='softmax'))
model_ff.summary()
model_ff.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
Total params: 5,560
Trainable params: 5,560
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_ff.fit(m_train, train_labels[index], epochs = 10)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 262us/sample - loss: 2.1909 - sparse_categorical_accuracy: 0.2140
Epoch 2/10
1000/1000 [==============================] - 0s 35us/sample - loss: 1.8331 - sparse_categorical_accuracy: 0.3930
Epoch 3/10
1000/1000 [==============================] - 0s 37us/sample - loss: 1.5682 - sparse_categorical_accuracy: 0.5420
Epoch 4/10
1000/1000 [==============================] - 0s 34us/sample - loss: 1.3996 - sparse_categorical_accuracy: 0.5840
Epoch 5/10
1000/1000 [==============================] - 0s 33us/sample - loss: 1.2564 - sparse_categorical_accuracy: 0.6410
Epoch 6/10
1000/1000 [==============================] - 0s 33us/sample - loss: 1.1737 - sparse_categorical_accuracy: 0.6630
Epoch 7/10
1000/1000 [==============================] - 0s 33us/sample - loss: 1.0732 - sparse_categorical_accuracy: 0.6910
Epoch 8/10
1000/1000 [==============================] - 0s 34us/sample - loss: 1.0093 - sparse_categorical_ac